In [ ]:
# Run these cells instead if you have 2 Factor Authentication enabled.
# After cloning the repo inside your Google Drive
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install tweet-preprocessor
! pip install demoji
! pip install googletrans
! pip install sentencepiececlean_text
! pip install sentencepiece

     |████████████████████████████████| 41 kB 454 kB/s 
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 6.5 MB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 65 kB 3.7 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15734 sha256=1b0816a1b1e1e1fe532e879903550de2a1e62671775625839c4081107d930398
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans
ERROR: Could not find a version that satisfies the requirement sentencepiececlean_text (from versions: none)
ERROR: No matching distribution found for sentencepiececlean_text
     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [ ]:
! pip install indic-nlp-library

     |████████████████████████████████| 40 kB 3.2 MB/s 
     |████████████████████████████████| 9.1 MB 7.1 MB/s 
     |████████████████████████████████| 548 kB 30.8 MB/s 
  Created wheel for sphinx-argparse: filename=sphinx_argparse-0.2.5-py3-none-any.whl size=11550 sha256=a510a322b3029b8369fdf0b31b303b2451cb173ec443ddc7b53c5041a7e14285
  Stored in directory: /root/.cache/pip/wheels/4a/4c/12/83c88bdc1bc352d7036de2891b5d6729adcf1bda3bab015560
Successfully built sphinx-argparse
  Attempting uninstall: docutils
    Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from sklearn import preprocessing
from tensorflow.keras.callbacks import Callback
import demoji
import preprocessor as preproc
import re
import string

In [ ]:
def remove_punctuation(text):
    return ''.join([c for c in text if c not in string.punctuation])

def remove_url(text):
    return re.sub(r'https?://\S+', ' ', text)

def remove_mention(text):
    return re.sub(r'@[A-Za-z0-9_]+', ' ', text)

def remove_hashtag(text):
    return re.sub(r'.*?(#\w+)|.+', ' ', text)  #TODO: check if using/removing the hashtag has any improvement

def remove_emoji(text):
    return demoji.replace(text, ' ')


def clean_row(text):
    return ' '.join(remove_punctuation(remove_url(remove_mention(remove_emoji(text)))).split()).lower()

def clean(df):
    df['clean_text'] = df.text.apply(clean_row)
    return df

In [ ]:
import os
os.chdir("/content/drive/My Drive/HASOC-2021/Subtask1-Marathi")

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('./Data/mr_Hasoc2021_train.xlsx')
df = clean(df)
df.head()

,text_id,text,task_1,clean_text
0,hasoc_mr_1,भारत 15 ऑगस्ट 1947 ला स्वतंत्र झाला आणि त्यानं...,NOT,भारत 15 ऑगस्ट 1947 ला स्वतंत्र झाला आणि त्यानं...
1,hasoc_mr_2,स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यास...,NOT,स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यास...
2,hasoc_mr_3,5 व्या नंबरची अर्थव्यवस्था आहे भारताची जगात 20...,NOT,5 व्या नंबरची अर्थव्यवस्था आहे भारताची जगात 20...
3,hasoc_mr_4,च्यायला म्हणजे दुबईचा फोन ही पुडीच निघाली की.,HOF,च्यायला म्हणजे दुबईचा फोन ही पुडीच निघाली की
4,hasoc_mr_5,ह्याला खरंतर कधीच आत टाकला पाहिजे होता. पैसा आ...,HOF,ह्याला खरंतर कधीच आत टाकला पाहिजे होता पैसा आण...


In [ ]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

In [ ]:
# Normalization
factory=IndicNormalizerFactory()
normalizer=factory.get_normalizer("mr")

num_rows = df.shape[0]
for i in range(num_rows):
    input_text = df["clean_text"][i]
    normalized_text = normalizer.normalize(input_text)
    # if (input_text != normalized_text):
    #     print("Original   ->",input_text)
    #     print("Normalized ->",normalized_text,"\n")
    df["text"][i] = normalized_text

In [ ]:
train=df.sample(frac=0.8,random_state=73)
test=df.drop(train.index)
train.to_csv('train_mar.csv')
test.to_csv('test_mar.csv')

In [ ]:
from collections import defaultdict as ddict 
import csv
label_dict1 = {
    'NOT':0,
    'HOF':1
}

def load_dataset(df="train_mar.csv"):
  lines = []
  with open(df, "r") as f:
    reader = csv.reader(f)
    for line in reader:
      lines.append(line)
    tweet=[]
    label1 =[]
    dic_data = ddict(list)
    for (i,line) in enumerate(lines):
      if i==0:
        continue

      _tweet = line[2]
      _label1 = line[3]

      tweet.append(_tweet)
      label1.append(label_dict1[_label1])
    label_list1 = ["HOF", "NOT"]
    dic_data['tweet']=tweet
    dic_data['label1']=label1
    return dic_data,label_list1

In [ ]:
train_data, label_list1= load_dataset()
# print(train_data['hypothesis'][:10])
print(train_data['label1'][:5])

[1, 0, 1, 0, 0]


In [ ]:
import torch
!pip install transformers -q
!pip install tokenizers -q
!pip install datasets
from transformers import BertTokenizer, AutoTokenizer
from transformers import BertForSequenceClassification, AdamW, AutoModelForSequenceClassification

     |████████████████████████████████| 2.6 MB 4.1 MB/s 
     |████████████████████████████████| 636 kB 50.4 MB/s 
     |████████████████████████████████| 3.3 MB 39.2 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 
     |████████████████████████████████| 264 kB 4.3 MB/s 
     |████████████████████████████████| 118 kB 47.9 MB/s 
     |████████████████████████████████| 243 kB 38.1 MB/s 


In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModelForSequenceClassification.from_pretrained('ai4bharat/indic-bert',num_labels=2)


Downloading:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['sop_classifier.classifier.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [ ]:
encoding = bert_tokenizer(train_data['tweet'] ,return_tensors='pt', padding=True)
print(encoding) 

{'input_ids': tensor([[     2,  26294,   4384,  ...,      0,      0,      0],
        [     2, 100705,    494,  ...,      0,      0,      0],
        [     2,   8351,   6676,  ...,      0,      0,      0],
        ...,
        [     2,  69847,   7175,  ...,      0,      0,      0],
        [     2,   2092,   2863,  ...,      0,      0,      0],
        [     2,   5478,   1976,  ...,      0,      0,      0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
print(len(encoding['attention_mask']))

1499


In [ ]:
class HASOC(torch.utils.data.Dataset):
  def __init__(self,encodings,labels):
    self.encodings=encodings
    self.labels = labels
  def __getitem__(self,idx):
    item ={key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
        return len(self.labels)

#train Data
Data = HASOC(encoding,train_data['label1']) # binary right now, if 4 way classification change in BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

test_data, label_test_list1 = load_dataset('test_mar.csv')
test_encoding = bert_tokenizer(test_data['tweet'] ,return_tensors='pt', padding=True)
test_Data = HASOC(test_encoding,test_data['label1'])


In [ ]:
from transformers import Trainer,TrainingArguments

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args =TrainingArguments(
    output_dir ='./output',
    num_train_epochs=8,
    per_device_train_batch_size =8,
    warmup_steps=500,
    logging_dir ='./logs',
    logging_steps=20,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model =model,
    args = training_args,
    train_dataset = Data,
    eval_dataset =test_Data,
    compute_metrics=compute_metrics


)
trainer.train()


Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

***** Running training *****
  Num examples = 1499
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1504
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Epoch,Training Loss,Validation Loss,Accuracy
1,0.664000,0.626006,0.664000
2,0.643400,0.627228,0.664000
3,0.643000,0.628617,0.704000
4,0.591900,0.583719,0.757333
5,0.499100,0.530468,0.770667
6,0.493200,0.481492,0.816000
7,0.401300,0.458305,0.842667
8,0.379200,0.448433,0.848000


***** Running Evaluation *****
  Num examples = 375
  Batch size = 8
***** Running Evaluation *****
  Num examples = 375
  Batch size = 8
Saving model checkpoint to ./output/checkpoint-500
Configuration saved in ./output/checkpoint-500/config.json
Model weights saved in ./output/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
***** Running Evaluation *****
  Num examples = 375
  Batch size = 8
***** Running Evaluation *****
  Num examples = 375
  Batch size = 8
***** Running Evaluation *****
  Num examples = 375
  Batch size = 8
Saving model checkpoint to ./output/checkpoint-1000
Configuration saved in ./output/checkpoint-1000/config.json
Model weights saved in ./output/checkpoint-1000/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/

TrainOutput(global_step=1504, training_loss=0.5462116591949412, metrics={'train_runtime': 209.6785, 'train_samples_per_second': 57.192, 'train_steps_per_second': 7.173, 'total_flos': 43099853116320.0, 'train_loss': 0.5462116591949412, 'epoch': 8.0})